In [21]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_colwidth', -1)
import os
from time import time
from pprint import pprint
import collections

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")
sns.set(font_scale=1.3)

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

import warnings
warnings.filterwarnings('ignore')

np.random.seed(37)

In [22]:
df_model = pd.read_pickle('../data/pickle_emotion/df_model_en_.p')

In [23]:
df_model.head()

,count_words,count_mentions,count_hashtags,count_capital_words,count_excl_quest_marks,count_urls,count_emojis,sentiment,origin_text,clean_text
0,22,5,2,1,1,0,0,joy,Happy to share my new video preview with some of my local fav's!\n#compose #excitement @CBCMusic @Musicyyc @CJSW @BreakOutWest @HifiClub,happy share new video preview local fav compose excitement
1,14,5,0,0,0,0,0,anticipation,@FieldYates @MatthewBerryTMR @Stephania_ESPN @MikeClayNFL @FrankCaliendo goddamn...the 'celebrity' draft at the end was classic.,goddamn celebrity draft end classic
2,11,0,0,0,0,0,0,pessimism,Dreams dashed and divided like million stars in the night sky.,dreams dashed divided like million stars night sky
3,17,0,0,0,0,0,1,joy,Accept the challenges so that you can literally even feel the exhilaration of victory.' -- George S. Patton 🐶,accept challenges literally even feel exhilaration victory george patton dogface
4,15,1,1,0,0,0,0,joy,Happy birthday to ma brother from another mother @Official1Cedi dis year More blessing #Break itz,happy birthday brother another mother dis year blessing break itz


In [24]:
X_train, X_test, y_train, y_test = train_test_split(df_model.drop('sentiment', axis=1), df_model.sentiment, test_size=0.1, random_state=37)

In [25]:
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X, **transform_params):
        return X[self.cols]

    def fit(self, X, y=None, **fit_params):
        return self

In [26]:
def grid_vect(clf, parameters_clf, X_train, X_test, parameters_text=None, vect=None):
    
    textcountscols = ['count_capital_words','count_emojis','count_excl_quest_marks','count_hashtags'
                      ,'count_mentions','count_urls','count_words']
    

    features = FeatureUnion([('textcounts', ColumnExtractor(cols=textcountscols))
                            , ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text')), ('vect', vect)]))]
                            , n_jobs=-1)

    
    pipeline = Pipeline([
        ('features', features)
        , ('clf', clf)
    ])
    
    # Join the parameters dictionaries together
    parameters = dict()
    if parameters_text:
        parameters.update(parameters_text)
    parameters.update(parameters_clf)

    # initiate gridsearchCV with parameters and pipline
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=5)
    
    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipeline.steps])
    print("parameters:")
    pprint(parameters)

    t0 = time()
    grid_search.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best CV score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    print("Test score with best_estimator_: %0.3f" % grid_search.best_estimator_.score(X_test, y_test))
    print("\n")
    print("Classification Report Test Data")
    print(classification_report(y_test, grid_search.best_estimator_.predict(X_test)))

    print("all results")
    means = grid_search.cv_results_['mean_test_score']
    stds = grid_search.cv_results_['std_test_score']
    params = grid_search.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))
    
    return grid_search

In [27]:
# Parameter grid settings for the vectorizers (Count and TFIDF)
parameters_vect = {
    'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
    'features__pipe__vect__ngram_range': ((1, 1), (1, 2)),
    'features__pipe__vect__min_df': (1,2)
}

# Parameter grid settings for MultinomialNB
parameters_mnb = {
    'clf__alpha': (0.25, 0.5, 0.75)
}

In [28]:
mnb = MultinomialNB()

In [29]:
countvect = CountVectorizer()

In [30]:
# MultinomialNB
best_mnb_countvect = grid_vect(mnb, parameters_mnb, X_train, X_test, parameters_text=parameters_vect, vect=countvect)

Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__alpha': (0.25, 0.5, 0.75),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   10.3s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   35.3s finished


done in 36.003s

Best CV score: 0.568
Best parameters set:
	clf__alpha: 0.75
	features__pipe__vect__max_df: 0.25
	features__pipe__vect__min_df: 2
	features__pipe__vect__ngram_range: (1, 1)
Test score with best_estimator_: 0.594


Classification Report Test Data
              precision    recall  f1-score   support

       anger       0.61      0.89      0.72       292
anticipation       0.24      0.11      0.15        76
     disgust       0.29      0.04      0.07        52
        fear       0.44      0.25      0.32        60
         joy       0.66      0.77      0.71       203
        love       0.00      0.00      0.00         4
    optimism       0.00      0.00      0.00        20
   pessimism       0.00      0.00      0.00        21
     sadness       0.00      0.00      0.00        16

   micro avg       0.59      0.59      0.59       744
   macro avg       0.25      0.23      0.22       744
weighted avg       0.50      0.59      0.52       744

all results
0.553481 (0.005900) w

In [31]:
tfidfvect = TfidfVectorizer()

In [32]:
best_mnb_tfidf = grid_vect(mnb, parameters_mnb, X_train, X_test, parameters_text=parameters_vect, vect=tfidfvect)

Performing grid search...
pipeline: ['features', 'clf']
parameters:
{'clf__alpha': (0.25, 0.5, 0.75),
 'features__pipe__vect__max_df': (0.25, 0.5, 0.75),
 'features__pipe__vect__min_df': (1, 2),
 'features__pipe__vect__ngram_range': ((1, 1), (1, 2))}
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   33.1s finished


done in 33.733s

Best CV score: 0.543
Best parameters set:
	clf__alpha: 0.25
	features__pipe__vect__max_df: 0.25
	features__pipe__vect__min_df: 2
	features__pipe__vect__ngram_range: (1, 1)
Test score with best_estimator_: 0.566


Classification Report Test Data
              precision    recall  f1-score   support

       anger       0.56      0.90      0.69       292
anticipation       0.20      0.05      0.08        76
     disgust       0.25      0.02      0.04        52
        fear       0.44      0.12      0.18        60
         joy       0.63      0.72      0.68       203
        love       0.00      0.00      0.00         4
    optimism       0.00      0.00      0.00        20
   pessimism       0.00      0.00      0.00        21
     sadness       0.00      0.00      0.00        16

   micro avg       0.57      0.57      0.57       744
   macro avg       0.23      0.20      0.19       744
weighted avg       0.46      0.57      0.48       744

all results
0.524649 (0.008566) w

In [33]:
#max_df: 0.25 or maximum document frequency of 25%.
#min_df: 2 or the words need to appear in at least 2 tweets
#ngram_range: (1, 1)
#clf__alpha: 0.75
textcountscols = ['count_capital_words','count_emojis','count_excl_quest_marks','count_hashtags'
                      ,'count_mentions','count_urls','count_words']
    
features = FeatureUnion([('textcounts', ColumnExtractor(cols=textcountscols))
                         , ('pipe', Pipeline([('cleantext', ColumnExtractor(cols='clean_text'))
                                              , ('vect', CountVectorizer(max_df=0.25, min_df=2, ngram_range=(1,1)))]))]
                       , n_jobs=-1)

pipeline = Pipeline([
    ('features', features)
    , ('clf', MultinomialNB(alpha=0.75))
])

best_model = pipeline.fit(df_model.drop('sentiment', axis=1), df_model.sentiment)

In [34]:
df_model_joy = pd.read_pickle('../data/df_model_joy.p')
best_model.predict(df_model_joy).tolist()

['joy', 'joy', 'anger']

In [35]:
df_model_anger = pd.read_pickle('../data/df_model_anger.p')
best_model.predict(df_model_anger).tolist()

['anger', 'anger', 'anger']

In [36]:
df_model_love = pd.read_pickle('../data/df_model_love.p')
best_model.predict(df_model_love).tolist()

['joy']

In [37]:
df_model_fear = pd.read_pickle('../data/df_model_fear.p')
best_model.predict(df_model_fear).tolist()

['anger']